In [47]:
import os
import time
import requests
import pandas as pd

api_key = os.environ.get('riot_api_key')
base_url = "https://na1.api.riotgames.com"
solo_duo = "RANKED_SOLO_5x5"
division = "EMERALD"

print(api_key)
print(base_url)

RGAPI-4fa3c214-6a7e-46b1-9a9b-b1caebf28d89
https://na1.api.riotgames.com


In [48]:
blue_team_id = 100
red_team_id = 200
def getWinner(teams):
	for t in teams:
		if t['teamId'] == blue_team_id:
			return 0 if t['win'] else 1

def getChampionID(participants, side, position):
	for p in participants:
		if p['teamId'] == side and p['teamPosition'] == position:
			return p['championId']
		
def getTeamGold(participants):
	blue_gold = 0
	red_gold = 0

	for p in participants:
		if p['teamId'] == blue_team_id:
			blue_gold += p['goldSpent']
		else:
			red_gold += p['goldSpent']
	
	return blue_gold, red_gold


def mapData(match_data):
	blue_gold, red_gold = getTeamGold(match_data['info']['participants'])

	blue_atakhan = 0
	red_atakhan = 0

	if 'atakhan' in match_data['info']['teams'][0]['objectives']:
		blue_atakhan = match_data['info']['teams'][0]['objectives']['atakhan']['kills']

	if 'atakhan' in match_data['info']['teams'][1]['objectives']:
		red_atakhan = match_data['info']['teams'][1]['objectives']['atakhan']['kills']

	
	scrubbed_match_data = {
		'match_id': match_data['metadata']['matchId'],
		'duration': match_data['info']['gameDuration'],
		'blue_top': getChampionID(match_data['info']['participants'], blue_team_id, 'TOP'),
		'blue_jg': getChampionID(match_data['info']['participants'], blue_team_id, 'JUNGLE'),
		'blue_mid': getChampionID(match_data['info']['participants'], blue_team_id, 'MIDDLE'),
		'blue_bot': getChampionID(match_data['info']['participants'], blue_team_id, 'BOTTOM'),
		'blue_supp': getChampionID(match_data['info']['participants'], blue_team_id, 'UTILITY'),
		'red_top': getChampionID(match_data['info']['participants'], red_team_id, 'TOP'),
		'red_jg': getChampionID(match_data['info']['participants'], red_team_id, 'JUNGLE'),
		'red_mid': getChampionID(match_data['info']['participants'], red_team_id, 'MIDDLE'),
		'red_bot': getChampionID(match_data['info']['participants'], red_team_id, 'BOTTOM'),
		'red_supp': getChampionID(match_data['info']['participants'], red_team_id, 'UTILITY'),
		'gold_difference': blue_gold - red_gold, # Negactive number means red side is ahead
		'blue_champ_kills': match_data['info']['teams'][0]['objectives']['champion']['kills'],
		'red_champ_kills': match_data['info']['teams'][1]['objectives']['champion']['kills'],
		'blue_tower_kills': match_data['info']['teams'][0]['objectives']['tower']['kills'],
		'red_tower_kills': match_data['info']['teams'][1]['objectives']['tower']['kills'],
		'blue_grubs': match_data['info']['teams'][0]['objectives']['horde']['kills'],
		'red_grubs': match_data['info']['teams'][1]['objectives']['horde']['kills'],
		'blue_heralds': match_data['info']['teams'][0]['objectives']['riftHerald']['kills'],
		'red_heralds': match_data['info']['teams'][1]['objectives']['riftHerald']['kills'],
		'blue_dragons': match_data['info']['teams'][0]['objectives']['dragon']['kills'],
		'red_dragons': match_data['info']['teams'][1]['objectives']['dragon']['kills'],
		'blue_barons': match_data['info']['teams'][0]['objectives']['baron']['kills'],
		'red_barons': match_data['info']['teams'][1]['objectives']['baron']['kills'],
		'blue_atakhan': blue_atakhan,
		'red_atakhan': red_atakhan,
		'winner': getWinner(match_data['info']['teams']) # 0 = Blue side, 1 = Red Side
	}

	return scrubbed_match_data

In [49]:
def getMatches(match_ids):
	all_matches = pd.DataFrame()
	bad_match_ids = []
	match_id = ""
	match_data = None
	try:

		for index, row in match_ids.iterrows():
			match_id = row.match_id

			url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
			response = requests.get(url)
			
			if response.status_code == 200:
				match_data = response.json()
			else:
				print(response)
				continue

			# did game end before 14 mins, if so skip
			if (match_data['info']['gameDuration'] < 840):
				bad_match_ids.append(match_id)
				continue

			# Check if it has grubs
			if ("horde" not in match_data['info']['teams'][1]['objectives']):
				bad_match_ids.append(match_id)
				continue

			match_data_dict = mapData(match_data)

			all_matches = pd.concat([all_matches, pd.DataFrame([match_data_dict])])
			if (index + 1) % 50 == 0:
				print(len(bad_match_ids))
				print(all_matches.shape)				
				time.sleep(90) # Sleep 90 seconds so that it won't hit the rate limit
	
	except Exception as e: 
		print(e)
		print(match_data)
	finally:
		all_matches.to_csv(f"./AllMatches2-{division}.csv")
		df = pd.DataFrame({ 'match_id': bad_match_ids })
		df.to_csv("./BadMatchIDs2.csv")

In [50]:
# Remove Duplicates
match_ids = pd.read_csv(f"MatchIDs-{division}.csv")
match_ids = match_ids.drop("Unnamed: 0", axis=1)

print(match_ids.shape)

match_ids = match_ids.drop_duplicates()

print(match_ids.shape)


(16375, 1)
(16094, 1)


In [51]:
bad_match_ids = pd.read_csv(f'./BadMatchIDs.csv')

match_ids = match_ids[~match_ids['match_id'].isin(bad_match_ids['match_id'])]

print(match_ids.shape)
match_ids.head()

(16034, 1)


,match_id
0,NA1_5244652123
1,NA1_5244623311
2,NA1_5244087574
3,NA1_5244059202
4,NA1_5244031543


In [52]:
all_matches_1 = pd.read_csv('./AllMatches.csv')
all_matches_2 = pd.read_csv('./AllMatches-EMERALD.csv')

all_matches = pd.concat([all_matches_1, all_matches_2])

match_ids = match_ids[~match_ids['match_id'].isin(all_matches['match_id'])]

print(match_ids.shape)

(9800, 1)


In [ ]:
getMatches(match_ids)

<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429